# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import sys
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key
print(geoapify_key)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


c7d27748d8124f94b33ba8697751eaa0


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barentu,15.1139,37.5928,18.95,87,98,2.78,ER,1693433482
1,1,kapuskasing,49.4169,-82.4331,19.77,49,55,3.20,CA,1693433309
2,2,lebu,-37.6167,-73.6500,9.97,73,38,1.80,CL,1693433483
3,3,port-aux-francais,-49.3500,70.2167,4.99,90,100,18.85,TF,1693433483
4,4,ushuaia,-54.8000,-68.3000,1.25,95,97,1.94,AR,1693433483


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
coordinates = {
    'latitude': city_data_df["Lat"], 
    'longitude': city_data_df["Lng"],
    'City': city_data_df["City"],
    'Humidity': city_data_df["Humidity"]
}
# Pandas Dataframe
coordinates_df = pd.DataFrame(coordinates)
coordinates_df
# Display the map
map_plot = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    width = 700,
    height = 500,
    scale = 0.8,
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp_condition = (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)
wind_speed_condition = city_data_df["Wind Speed"] < 4.5
cloudiness_condition = city_data_df["Cloudiness"] == 0
ideal_weather_df = city_data_df[max_temp_condition & wind_speed_condition & cloudiness_condition]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()
# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
78,78,east london,-33.0153,27.9116,21.09,41,0,3.81,ZA,1693433500
207,207,chamical,-30.3600,-66.3140,24.48,19,0,4.06,AR,1693433522
243,243,ayancik,41.9447,34.5861,25.76,58,0,1.41,TR,1693433531
248,248,melilla,35.2937,-2.9383,25.21,61,0,3.09,ES,1693433273
257,257,ovalle,-30.5983,-71.2003,23.02,30,0,2.00,CL,1693433353


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
78,east london,ZA,-33.0153,27.9116,41,
207,chamical,AR,-30.3600,-66.3140,19,
243,ayancik,TR,41.9447,34.5861,58,
248,melilla,ES,35.2937,-2.9383,61,
257,ovalle,CL,-30.5983,-71.2003,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"
params = {
    "limit": limit, 
    "apiKey": geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")
print("----------------------------------------")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
----------------------------------------
east london - nearest hotel: No hotel found
chamical - nearest hotel: No hotel found
ayancik - nearest hotel: No hotel found
melilla - nearest hotel: Rusadir
ovalle - nearest hotel: Hotel Roxy
barra dos coqueiros - nearest hotel: Grande Hotel
prairie du chien - nearest hotel: River Distrit Hotel
shahba - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
ceuta - nearest hotel: Ulises
shawano - nearest hotel: No hotel found
deschutes river woods - nearest hotel: WorldMark Bend - Seventh Mountain Resort
korla - nearest hotel: Silver Star Hotel
tostado - nearest hotel: Hotel Centro
holland - nearest hotel: Haworth Hotel
langzhong - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
78,east london,ZA,-33.0153,27.9116,41,No hotel found
207,chamical,AR,-30.3600,-66.3140,19,No hotel found
243,ayancik,TR,41.9447,34.5861,58,No hotel found
248,melilla,ES,35.2937,-2.9383,61,Rusadir
257,ovalle,CL,-30.5983,-71.2003,30,Hotel Roxy


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.8,
    color = "City",
    hover_cols = ["City", "Country", "Hotel Name"]
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)

In [8]:
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
78,east london,ZA,-33.0153,27.9116,41,No hotel found
207,chamical,AR,-30.3600,-66.3140,19,No hotel found
243,ayancik,TR,41.9447,34.5861,58,No hotel found
248,melilla,ES,35.2937,-2.9383,61,Rusadir
257,ovalle,CL,-30.5983,-71.2003,30,Hotel Roxy
281,barra dos coqueiros,BR,-10.9089,-37.0386,78,Grande Hotel
292,prairie du chien,US,43.0517,-91.1412,27,River Distrit Hotel
306,shahba,SY,32.8551,36.6290,44,No hotel found
319,port alfred,ZA,-33.5906,26.8910,39,No hotel found
373,ceuta,ES,35.8903,-5.3075,74,Ulises
